In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# training script
script_dir = prefix.joinpath("code", "lightgbm", "iris")
script_name = "train.py"

# environment file
environment_file = prefix.joinpath("environments", "lightgbm.txt")

# azure ml settings
environment_name = "hpo-lightgbm"
experiment_name = "hpo-lightgbm-test"
compute_target = "local"

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment

ds = ws.datasets["iris"]

env = Environment.from_pip_requirements(environment_name, environment_file)

arguments = [
    "--data-dir",
    ds.as_mount(),
    "--num-boost-round",
    10,
    "--booster",
    "gbdt",
    "--num-iterations",
    10,
    "--num-leaves",
    16,
    "--num-threads",
    0,
    "--learning-rate",
    0.01,
    "--seed",
    42,
]

src = ScriptRunConfig(
    source_directory=script_dir,
    script=script_name,
    arguments=arguments,
    environment=env,
    compute_target=compute_target,
)

run = Experiment(ws, experiment_name).submit(src)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
run.wait_for_completion()
metrics = run.get_metrics()

training_time = metrics["training_time"]
loss = metrics["loss"]
acc = metrics["accuracy"]

training_time, loss, acc